## Импорты, параметры и константы

In [1]:
import os
import json
from pathlib import Path
from collections import Counter
import time
import sys

import numpy as np
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.models import mobilenet_v3_small, MobileNet_V3_Small_Weights
from torchvision.models import mobilenet_v3_large, MobileNet_V3_Large_Weights

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import psutil
from tqdm.notebook import tqdm


DEFAULT_TRAIN_CSV = "data/scooters_label_train.csv"
DEFAULT_TEST_CSV  = "data/scooters_label_test.csv"
OUTPUT_DIR = "MobileNetV3"
IMG_SIZE = 224
BATCH_SIZE = 64
NUM_WORKERS = min(8, os.cpu_count())
NUM_EPOCHS = 100
LR = 5e-4
WEIGHT_DECAY = 1e-5
PATIENCE = 30
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
NUM_CLASSES = 3
SEED = 42
PRETRAINED = True
DATA_FRACTION = 0.6 
MIN_FREE_GB = 5.0

## Архитектура (модель, агументация, загрузка данных и т.д.)

In [2]:
def seed_everything(seed=SEED):
    import random
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

class RandomRotate90:
    """Случайный поворот на 0, 90, 180 или 270 градусов"""
    def __call__(self, img):
        angle = np.random.choice([0, 90, 180, 270])
        return img.rotate(angle) if angle != 0 else img


class ScooterDataset(Dataset):
    def __init__(self, csv_file, class_to_idx=None, transforms=None, sample_frac=1.0, cache=False, min_free_gb=1.0, max_cache_images = 21500):
        if isinstance(csv_file, pd.DataFrame):
            self.df = csv_file.copy()
        else:
            self.df = pd.read_csv(csv_file)

        if 0 < sample_frac < 1.0:
            self.df = self.df.sample(frac=sample_frac, random_state=SEED).reset_index(drop=True)

        self.labels = self.df["scooter"].astype(str).tolist()
        self.paths = self.df["path"].tolist()

        if class_to_idx is None:
            classes = sorted(list(set(self.labels)))
            self.class_to_idx = {c: i for i, c in enumerate(classes)}
        else:
            self.class_to_idx = class_to_idx

        self.targets = [self.class_to_idx[l] for l in self.labels]
        self.transforms = transforms
        self.cache = cache
        self.min_free_gb = min_free_gb
        self.cached_images = {}
        self.max_cache_images = max_cache_images

        if self.cache:
            self._cache_images_safely()

    def _cache_images_safely(self):
        total_cached = 0
        for idx, p in enumerate(tqdm(self.paths, mininterval=3, desc="Caching images")):
            if total_cached >= self.max_cache_images:
                print(f"Reached max_cache_images limit: {self.max_cache_images}")
                break
                
            mem = psutil.virtual_memory()
            free_gb = mem.available / 1e9
            if free_gb < self.min_free_gb:
                print(f"Stopping caching: only {free_gb:.2f} GB free after {total_cached} images")
                break

            try:
                img = Image.open(p).convert("RGB")
            except:
                img = Image.new("RGB", (IMG_SIZE, IMG_SIZE), (0, 0, 0))

            if self.transforms:
                img = self.transforms(img)

            self.cached_images[idx] = img
            total_cached += 1

        print(f"Cached {total_cached} / {len(self.paths)} images")

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        label = self.targets[idx]
        if idx in self.cached_images:
            img = self.cached_images[idx]
        else:
            p = self.paths[idx]
            try:
                img = Image.open(p).convert("RGB")
            except:
                img = Image.new("RGB", (IMG_SIZE, IMG_SIZE), (0, 0, 0))
            if self.transforms:
                img = self.transforms(img)
        return img, label


def get_transforms(img_size=IMG_SIZE):
    train_transforms = transforms.Compose([
        RandomRotate90(),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomApply([transforms.ColorJitter(brightness=0.3, contrast=0.3)], p=0.7),
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
    ])
    val_transforms = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
    ])
    return train_transforms, val_transforms


def build_model(num_classes=NUM_CLASSES, pretrained=True):
    weights = MobileNet_V3_Small_Weights.IMAGENET1K_V1 if pretrained else None
    model = mobilenet_v3_small(weights=weights)
    if hasattr(model, 'classifier'):
        in_features = model.classifier[-1].in_features
        model.classifier[-1] = nn.Linear(in_features, num_classes)
    return model


def evaluate(model, dataloader, criterion, device, log=True):
    model.eval()
    all_preds, all_targets = [], []
    running_loss = 0.0
    inference_times = []

    iterator = tqdm(dataloader, desc="Evaluating", leave=False) if log else dataloader

    with torch.no_grad():
        for imgs, labels in iterator:
            imgs = imgs.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)

            start = time.time()
            outputs = model(imgs)
            inference_times.append(time.time() - start)

            loss = criterion(outputs, labels)
            running_loss += loss.item() * imgs.size(0)

            preds = outputs.argmax(dim=1).cpu().numpy()
            all_preds.extend(preds.tolist())
            all_targets.extend(labels.cpu().numpy().tolist())

    avg_loss = running_loss / len(dataloader.dataset)
    avg_time_per_image = np.mean(inference_times) / imgs.size(0)
    total_time_per_image = np.sum(inference_times) / len(dataloader.dataset)

    acc = accuracy_score(all_targets, all_preds)
    prec = precision_score(all_targets, all_preds, average='macro', zero_division=0)
    rec = recall_score(all_targets, all_preds, average='macro', zero_division=0)
    f1 = f1_score(all_targets, all_preds, average='macro', zero_division=0)
    cm = confusion_matrix(all_targets, all_preds)

    print(f"Average inference time per image: {total_time_per_image*1000:.2f} ms")

    return {
        'loss': avg_loss, 'accuracy': acc, 'precision': prec, 'recall': rec, 'f1': f1,
        'confusion_matrix': cm, 'avg_inference_time': total_time_per_image
    }


def evaluate_test(model, dataloader, criterion, device, class_names, log=True):
    model.eval()
    all_preds, all_targets = [], []
    running_loss = 0.0
    inference_times = []

    iterator = tqdm(dataloader, desc="Evaluating on TEST set", leave=False) if log else dataloader

    with torch.no_grad():
        for imgs, labels in iterator:
            imgs = imgs.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)

            start = time.time()
            outputs = model(imgs)
            inference_times.append(time.time() - start)

            loss = criterion(outputs, labels)
            running_loss += loss.item() * imgs.size(0)

            preds = outputs.argmax(dim=1).cpu().numpy()
            all_preds.extend(preds.tolist())
            all_targets.extend(labels.cpu().numpy().tolist())

    avg_loss = running_loss / len(dataloader.dataset)
    acc = accuracy_score(all_targets, all_preds)
    prec = precision_score(all_targets, all_preds, average='macro', zero_division=0)
    rec = recall_score(all_targets, all_preds, average='macro', zero_division=0)
    f1 = f1_score(all_targets, all_preds, average='macro', zero_division=0)
    cm = confusion_matrix(all_targets, all_preds)

    print("\n---------------- Final TEST Evaluation ----------------")
    print(f"Loss:      {avg_loss:.4f}")
    print(f"Accuracy:  {acc:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"Recall:    {rec:.4f}")
    print(f"F1-score:  {f1:.4f}")
    print("\nConfusion Matrix:")
    print(cm)

    report = classification_report(
        all_targets, all_preds, 
        target_names=class_names, 
        output_dict=True, 
        zero_division=0
    )

    df_report = pd.DataFrame(report).transpose()
    print("\nPer-class metrics:")
    print(df_report.round(4))

    avg_time_per_image = np.sum(inference_times) / len(dataloader.dataset)
    print(f"\nAverage inference time per image: {avg_time_per_image*1000:.2f} ms")

    return {
        'loss': avg_loss,
        'accuracy': acc,
        'precision': prec,
        'recall': rec,
        'f1': f1,
        'confusion_matrix': cm,
        'per_class': df_report,
        'avg_inference_time': avg_time_per_image
    }


def train_loop(train_loader, val_loader, model, criterion, optimizer, device, num_epochs=NUM_EPOCHS):
    best_val_f1 = -float('inf')
    best_epoch = -1
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    for epoch in range(1, num_epochs+1):
        model.train()
        running_loss = 0.0
        total_cpu_time = 0.0
        total_gpu_time = 0.0

        pbar = tqdm(train_loader, desc=f"Epoch {epoch}/{num_epochs}")
        for imgs, labels in pbar:
            start_cpu = time.time()
            imgs = imgs.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)
            total_cpu_time += time.time() - start_cpu

            start_gpu = time.time()
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_gpu_time += time.time() - start_gpu

            running_loss += loss.item() * imgs.size(0)
            processed = min(((pbar.n + 1) * imgs.size(0)), len(train_loader.dataset))
            avg_loss_running = running_loss / processed
            pbar.set_postfix({'loss': f"{avg_loss_running:.4f}"})

        epoch_train_loss = running_loss / len(train_loader.dataset)

        # Валидация
        start_val = time.time()
        val_res = evaluate(model, val_loader, criterion, device, False)
        val_time = time.time() - start_val

        total_time = total_cpu_time + total_gpu_time + val_time
        cpu_percent = total_cpu_time / total_time * 100
        gpu_percent = total_gpu_time / total_time * 100
        val_percent = val_time / total_time * 100

        print(f"Epoch {epoch} summary: total_time={total_time:.1f}s, CPU={cpu_percent:.1f}%, GPU={gpu_percent:.1f}%, VAL={val_percent:.1f}%")
        print(f"Train loss={epoch_train_loss:.4f}, Val loss={val_res['loss']:.4f}, Val f1={val_res['f1']:.4f}, Val acc={val_res['accuracy']:.4f}")

        if val_res['f1'] > best_val_f1:
            best_val_f1 = val_res['f1']
            best_epoch = epoch
            ckpt_path = os.path.join(OUTPUT_DIR, 'mobilenetv3_large_best.pth')
            torch.save({'epoch': epoch, 'model_state': model.state_dict(), 'optimizer_state': optimizer.state_dict()}, ckpt_path)
            print(f"Saved best model to {ckpt_path}")

        if epoch - best_epoch >= PATIENCE:
            print(f"Early stopping triggered")
            break

    print(f"Training finished. Best epoch: {best_epoch}")

## Загрузка данных и кэширование

In [3]:
seed_everything(SEED)
if DEVICE == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")

train_transforms, val_transforms = get_transforms(IMG_SIZE)

full_train_df = pd.read_csv(DEFAULT_TRAIN_CSV)
train_df, val_df = train_test_split(
    full_train_df, 
    test_size=0.1, 
    random_state=SEED, 
    stratify=full_train_df['scooter']
)

test_df = pd.read_csv(DEFAULT_TEST_CSV)

if 0 < DATA_FRACTION < 1.0:
    train_df = train_df.sample(frac=DATA_FRACTION, random_state=SEED).reset_index(drop=True)
    val_df = val_df.sample(frac=DATA_FRACTION, random_state=SEED).reset_index(drop=True)
    test_df  = test_df.sample(frac=DATA_FRACTION, random_state=SEED).reset_index(drop=True)

classes = sorted(train_df['scooter'].unique())
class_to_idx = {c: i for i, c in enumerate(classes)}
print("Class to idx:", class_to_idx)

train_dataset = ScooterDataset(train_df, class_to_idx=class_to_idx, transforms=train_transforms, cache=True, min_free_gb=MIN_FREE_GB)
val_dataset   = ScooterDataset(val_df,   class_to_idx=class_to_idx, transforms=val_transforms,   cache=True, min_free_gb=MIN_FREE_GB)
test_dataset  = ScooterDataset(test_df, class_to_idx=class_to_idx, transforms=val_transforms, cache=False)

print(f"Train samples: {len(train_dataset)}, Val samples: {len(val_dataset)}, Test samples: {len(test_dataset)}")

counts = Counter(train_dataset.targets)
class_counts = [counts[i] for i in range(len(class_to_idx))]
total = sum(class_counts)
class_weights = [total/(len(class_counts)*c) if c>0 else 0.0 for c in class_counts]
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(DEVICE)

pin_memory = DEVICE == "cuda"
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,  num_workers=NUM_WORKERS, pin_memory=pin_memory)
val_loader   = DataLoader(val_dataset,   batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=pin_memory)
test_loader  = DataLoader(test_dataset,  batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=pin_memory)

GPU: NVIDIA L4
Class to idx: {'absent': 0, 'part': 1, 'present': 2}


Caching images:   0%|          | 0/21169 [00:00<?, ?it/s]

Reached max_cache_images limit: 20000
Cached 20000 / 21169 images


Caching images:   0%|          | 0/2353 [00:00<?, ?it/s]

Cached 2353 / 2353 images
Train samples: 21169, Val samples: 2353, Test samples: 5881


## Обучение

In [13]:
model = build_model(num_classes=len(class_to_idx), pretrained=PRETRAINED).to(DEVICE)
criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)

train_loop(train_loader, val_loader, model, criterion, optimizer, DEVICE, NUM_EPOCHS)

Epoch 1/100:   0%|          | 0/331 [00:00<?, ?it/s]

Average inference time per image: 0.11 ms
Epoch 1 summary: total_time=12.5s, CPU=1.2%, GPU=76.2%, VAL=22.6%
Train loss=0.3837, Val loss=0.3144, Val f1=0.8749, Val acc=0.9108
Saved best model to MobileNetV3/mobilenetv3_large_best.pth


Epoch 2/100:   0%|          | 0/331 [00:00<?, ?it/s]

Average inference time per image: 0.10 ms
Epoch 2 summary: total_time=12.3s, CPU=0.9%, GPU=76.2%, VAL=22.9%
Train loss=0.2489, Val loss=0.3220, Val f1=0.8198, Val acc=0.8627


Epoch 3/100:   0%|          | 0/331 [00:00<?, ?it/s]

...

Average inference time per image: 0.10 ms
Epoch 98 summary: total_time=12.5s, CPU=1.1%, GPU=76.3%, VAL=22.6%
Train loss=0.0129, Val loss=0.9981, Val f1=0.8829, Val acc=0.9188


Epoch 99/100:   0%|          | 0/331 [00:00<?, ?it/s]

Average inference time per image: 0.10 ms
Epoch 99 summary: total_time=12.5s, CPU=0.9%, GPU=76.6%, VAL=22.5%
Train loss=0.0107, Val loss=0.8228, Val f1=0.8704, Val acc=0.9074


Epoch 100/100:   0%|          | 0/331 [00:00<?, ?it/s]

Average inference time per image: 0.10 ms
Epoch 100 summary: total_time=12.8s, CPU=1.0%, GPU=76.7%, VAL=22.3%
Train loss=0.0147, Val loss=1.3102, Val f1=0.8754, Val acc=0.9163
Training finished. Best epoch: 47


## Оценка на test

In [14]:
best_ckpt = os.path.join(OUTPUT_DIR, 'mobilenetv3_large_best.pth')

if os.path.exists(best_ckpt):
    checkpoint = torch.load(best_ckpt, map_location=DEVICE)
    model.load_state_dict(checkpoint['model_state'])

    print("\n---------------- Final evaluation on TEST set ----------------")
    test_results = evaluate_test(model, test_loader, criterion, DEVICE, class_names=list(class_to_idx.keys()))
else:
    print("No checkpoint found.")


---------------- Final evaluation on TEST set ----------------


Evaluating on TEST set:   0%|          | 0/92 [00:00<?, ?it/s]


---------------- Final TEST Evaluation ----------------
Loss:      0.7898
Accuracy:  0.9160
Precision: 0.8668
Recall:    0.8882
F1-score:  0.8771

Confusion Matrix:
[[ 449   21   18]
 [  33  849  178]
 [  45  199 4089]]

Per-class metrics:
              precision  recall  f1-score   support
absent           0.8520  0.9201    0.8847   488.000
part             0.7942  0.8009    0.7976  1060.000
present          0.9543  0.9437    0.9489  4333.000
accuracy         0.9160  0.9160    0.9160     0.916
macro avg        0.8668  0.8882    0.8771  5881.000
weighted avg     0.9169  0.9160    0.9163  5881.000

Average inference time per image: 0.13 ms
